In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler      # better for heavy outliers
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score


/var/folders/69/s6p755q576qb2bbk9bc__tr40000gn/T/ipykernel_1159/3635397593.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# import transaction train data
train_trns = pd.read_csv('/Users/oskarwallberg/desktop/kaggle-datasets/ieee-fraud-detection/train_transaction.csv')

In [3]:
V_COLS = ['V'+str(n) for n in range(1, 340)]
train_trns
train_trns.shape

(590540, 394)

In [4]:
df_vesta = train_trns[V_COLS].copy()
df_vesta.shape

(590540, 339)

In [5]:
df_vesta = df_vesta.fillna(0)
df_vesta.isna().any().any()

False

In [6]:
pca = PCA(n_components=20)

In [7]:
# df_vesta_pca = pd.DataFrame(data=pca.fit_transform(X=df_vesta))
# df_vesta_pca.shape

In [8]:
# np.cumsum(pca.explained_variance_ratio_)

In [9]:
TARGET_FEATURE = 'isFraud'
train_trns[TARGET_FEATURE].isna().any()

False

In [10]:
X = df_vesta
y = train_trns[TARGET_FEATURE]
X.shape, y.shape

((590540, 339), (590540,))

In [11]:
train_trns.shape[0] / train_trns['isFraud'].sum()

28.579586700866283

In [12]:
rb = RobustScaler(with_centering=True, with_scaling=True)

xgbclf = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.1,
    scale_pos_weight=4, # <- for class imbalance
)

N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True)

In [13]:
scores = np.zeros(N_FOLDS)

# NOTE about 0.7% of values are flagged as fraudulent per fold
for i, (train_idx, val_idx) in enumerate(skf.split(X=X, y=y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    X_train_sc = rb.fit_transform(X=X_train)
    X_val_sc = rb.transform(X=X_val)

    xgbclf.fit(X=X_train_sc, y=y_train, ) # eval_set=[(X_val_sc, y_val)]
    y_pred = xgbclf.predict(X=X_val_sc)
    y_score: np.ndarray = xgbclf.predict_proba(X=X_val_sc)[:, 1]
    
    print(f'Fold {i+1}/{N_FOLDS}')
    print(classification_report(y_true=y_val, y_pred=y_pred))
    print(f'ROC-AUC score: {roc_auc_score(y_true=y_val, y_score=y_score)}')

Fold 1/5
              precision    recall  f1-score   support

           0       0.98      0.99      0.99    113976
           1       0.68      0.41      0.51      4132

    accuracy                           0.97    118108
   macro avg       0.83      0.70      0.75    118108
weighted avg       0.97      0.97      0.97    118108

ROC-AUC score: 0.8648530972468789
Fold 2/5
              precision    recall  f1-score   support

           0       0.98      0.99      0.99    113976
           1       0.68      0.42      0.52      4132

    accuracy                           0.97    118108
   macro avg       0.83      0.71      0.75    118108
weighted avg       0.97      0.97      0.97    118108

ROC-AUC score: 0.8615284855404418
Fold 3/5
              precision    recall  f1-score   support

           0       0.98      0.99      0.99    113975
           1       0.70      0.43      0.53      4133

    accuracy                           0.97    118108
   macro avg       0.84      0.71

In [18]:
roc_auc_score(y_true=y, y_score=0.7*np.ones(y.shape[0]))

0.5